## Importando librerias

In [1]:
import pandas as pd
import generadoDF
import numpy as np
from pyscipopt import Model,quicksum
from sklearn.neighbors import BallTree

## Demanda

In [2]:
# Llamando datos de demanda
dfNodoNoA = pd.read_csv('nodosnoabastecidos.csv', usecols=['y','x'])

In [3]:
#indexes de demanda (sirve para la implementacion de la optimizacion del final)
I =[ i for i in range(len(dfNodoNoA))]

In [4]:
#-----
# Implementación de BALL TREE: Algoritmo que permite hallar lo k puntos vecinos
#-----


##INPUTS

#Datos de donde se quiere buscar los k vecinos y obtener sus poblaciones
dfSource = generadoDF.DfGenerator('dflatlongLima.csv')


#volviendo numpy los df anteriores para BallTree (el método recibe numpy)
matrizSource = pd.DataFrame.to_numpy(dfSource[['latitude', 'longitude']])
matrizNoAbas = pd.DataFrame.to_numpy(dfNodoNoA)


In [5]:
#IMPLEMENTACIÓN

#buscando los 10 vecinos más cercanos a los nodos no abastecidos con BallTree
tree = BallTree(matrizSource)              
dist, ind = tree.query(matrizNoAbas, k=10)


#convirtiendo en DF los índices de los puntos más cercanos
DFmatrixIndicesCercanos=pd.DataFrame(ind)

In [6]:
#Creando una matriz con las poblaciones asociadas a los indexes obtenidos antes

MatrizPopulCercanos = pd.DataFrame(index=DFmatrixIndicesCercanos.index, 
             columns=DFmatrixIndicesCercanos.columns)

for columna in DFmatrixIndicesCercanos.columns:
    for fila in DFmatrixIndicesCercanos.index:
        MatrizPopulCercanos[columna][fila] = dfSource['population_2020'][DFmatrixIndicesCercanos[columna][fila]]


#Sumando todas las filas para obtener demanda de los 10 vecinos cercanos a un NoAbaste
suma = MatrizPopulCercanos.sum(axis=1)

In [7]:
#creando diccionario con demandas de agua (servira para la optimizacion al final)
#LA DEMANDA ES LA CANTIDAD DE PERSONAS * 20 LITROS Q CONSUME CADA UNA

litrosDemandaxPersona = 20

d = { i:round(suma[i]*litrosDemandaxPersona,2) for i in range(0, len(I) )}

## Capacidad

In [8]:
#indexes de capacidad (sirve para la implementacion de la optimizacion del final)
J =[ i for i in range(22)] #Recordar que son 22 los surtidores de agua


In [50]:
#Capacidad de camiones
#Funcion: capacidad = 10,000litros * 15cantidad de camiones * 8veces

capacidad = 10000*15*8

In [51]:
#creando diccionario con capacidades de surtidores

M = {x:capacidad for x in range(len(J))}

## Costo

Para los costos se tiene la siguiente forma de diccionario:

c = {(1,1):4,    (1,2):6,    (1,3):9,
     (2,1):5,    (2,2):4,    (2,3):7,
     (3,1):6,    (3,2):3,    (3,3):3,
     (4,1):8,    (4,2):5,    (4,3):3,
     (5,1):10,   (5,2):8,    (5,3):4,
     }

In [21]:
### Formando los Pares: [Surtidor, Demanda]
paresSurtiDemand = [[a, b] for a in J for b in I]


###Formando [Costo]

#Subiendo archivo de Costos Individuales. 
#RECUERDA PONER EL NOMBRES según el escenario que se usas. Para este caso es distancia
dfCostosFinal=generadoDF.DfGenerator('dfCostoInvidualesDISTANCIA.csv')

### Generando pares de costos (c)

In [22]:
#agrepando[Surtidor,Demanda,Costos]
ParesSurtiDemanCost = np.hstack([paresSurtiDemand,dfCostosFinal])

#reordenando el array. En la libreria pide [Demanda,Surtido,Costos]
i=[1,0,2]
ParesDemanSurtiCost = ParesSurtiDemanCost[:,i]

In [23]:
#Formando los pares
c = {(a,b):c for a,b,c in ParesDemanSurtiCost}

## Optimización

In [52]:
model = Model("transportation")
x = {}
for i in I:
    for j in J:
        x[i,j] = model.addVar(vtype="C", name="x(%s,%s)" % (i,j))

In [53]:
for i in I:
    model.addCons(quicksum(x[i,j] for j in J if (i,j) in x) == d[i], name="Demand(%s)" % i)

In [54]:
for j in J:
    model.addCons(quicksum(x[i,j] for i in I if (i,j) in x) <= M[j], name="Capacity(%s)" % j)

In [55]:
model.setObjective(quicksum(c[i,j]*1  for (i,j) in c), "minimize")

In [60]:
model.optimize()
print("Valor optimo", model.getObjVal())
EPS = 1.e-6
for (i,j) in x:
    if model.getVal(x[i,j]) > EPS:
        print("Enviando %10s litros del surtidor %3s a la familia %3s" % (model.getVal(x[i,j]),j,i))

('Valor optimo', 3296528466.7900047)
Enviando    9970.05 litros del surtidor  13 a la familia 3336
Enviando     1361.9 litros del surtidor  12 a la familia 285
Enviando    6820.68 litros del surtidor   0 a la familia 1414
Enviando    9970.05 litros del surtidor   4 a la familia 3530
Enviando    9970.05 litros del surtidor   2 a la familia 726
Enviando    3686.74 litros del surtidor   9 a la familia 2196
Enviando    3686.74 litros del surtidor   7 a la familia 1262
Enviando    2857.78 litros del surtidor  16 a la familia 3643
Enviando    9970.05 litros del surtidor  13 a la familia 889
Enviando    9970.05 litros del surtidor  21 a la familia 3280
Enviando    2521.98 litros del surtidor  12 a la familia 2095
Enviando    6820.68 litros del surtidor  21 a la familia 1196
Enviando    6820.68 litros del surtidor   0 a la familia 246
Enviando     4053.7 litros del surtidor  21 a la familia 425
Enviando    9970.05 litros del surtidor  16 a la familia 4153
Enviando    2521.98 litros del surtido

Enviando    2857.78 litros del surtidor  10 a la familia 3264
Enviando    2857.78 litros del surtidor  10 a la familia 3638
Enviando    2857.78 litros del surtidor  20 a la familia 3172
Enviando    9970.05 litros del surtidor   3 a la familia 3012
Enviando    2857.78 litros del surtidor   9 a la familia 3620
Enviando    2857.78 litros del surtidor  10 a la familia 3273
Enviando   10034.02 litros del surtidor   0 a la familia 2742
Enviando    3686.74 litros del surtidor  21 a la familia 1327
Enviando    2857.78 litros del surtidor  10 a la familia 3387
Enviando    2857.78 litros del surtidor  10 a la familia 3383
Enviando    2857.78 litros del surtidor   8 a la familia 3814
Enviando    6820.68 litros del surtidor   0 a la familia 1525
Enviando   10034.02 litros del surtidor  21 a la familia 2702
Enviando    2857.78 litros del surtidor  10 a la familia 3772
Enviando    3686.74 litros del surtidor  13 a la familia 1913
Enviando    2521.98 litros del surtidor  13 a la familia 1999
Enviando

Enviando    6820.68 litros del surtidor   5 a la familia 1559
Enviando    4626.92 litros del surtidor  15 a la familia 1183
Enviando    3686.74 litros del surtidor   9 a la familia 2269
Enviando    2857.78 litros del surtidor  20 a la familia 3425
Enviando    9970.05 litros del surtidor  12 a la familia 605
Enviando    9970.05 litros del surtidor   4 a la familia 3550
Enviando    9970.05 litros del surtidor  21 a la familia 320
Enviando    8657.69 litros del surtidor   4 a la familia 2596
Enviando    9970.05 litros del surtidor  15 a la familia 588
Enviando    3686.74 litros del surtidor  17 a la familia  69
Enviando    3686.74 litros del surtidor  19 a la familia 1882
Enviando    2857.78 litros del surtidor  19 a la familia 3730
Enviando    3686.74 litros del surtidor  21 a la familia 2462
Enviando    2521.98 litros del surtidor  13 a la familia 1868
Enviando    2857.78 litros del surtidor  10 a la familia 3913
Enviando    3686.74 litros del surtidor   7 a la familia 261
Enviando    9